# Reconstruction of a complex wavefunction

In this tutorial, a walkthrough of how to reconstruct a **complex** wavefunction via training a *Restricted Boltzmann Machine* (RBM), the neural network behind qucumber, will be presented. After the training process, this tutorial will also demonstrate how new data can be generated.


## The wavefunction to be reconstructed
The simple wavefunction below describing two qubits (coefficients stored in *qubits_psi.txt*) will be reconstructed.

\begin{equation}
            \vert\psi \rangle = \alpha \vert00\rangle + \beta \vert 01\rangle + \gamma \vert10\rangle + \delta \vert11\rangle
\end{equation}

where the exact values of $\alpha, \beta, \gamma$ and $\delta$ used for this tutorial are 

\begin{align}
\alpha & = 0.2861  + 0.0539 i \\
\beta &  = 0.3687 - 0.3023 i \\
\gamma & = -0.1672 - 0.3529 i \\
\delta & = -0.5659 - 0.4639 i.
\end{align}

Our example dataset, *qubits_train.txt*, comprises of 500 $\sigma$ measurements made in various bases (X, Y and Z). A corresponding file containing the bases for each data point in *qubits_train.txt*, *qubits_bases.txt*, is also required. As per convention, spins are represented in binary notation with zero and one denoting spin-down and spin-up, respectively.

## Using qucumber 

### Imports
To begin the tutorial, we first import the required Python packages.

In [1]:
import numpy as np
import torch

from qucumber.nn_states import ComplexWavefunction

from qucumber.callbacks import MetricEvaluator

import qucumber.utils.unitaries as unitaries

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data

The Python class *ComplexWavefunction* contains generic properties of a RBM meant to reconstruct a complex wavefunction, the most notable one being the gradient function required for stochastic gradient descent.

To instantiate a *ComplexWavefunction* object, one needs to specify the number of visible and hidden units in the RBM. The number of visible units, *num_visible*, is given by the size of the physical system, i.e. the number of spins or qubits (10 in our case), while the number of hidden units, *num_hidden*, can be varied to change the expressiveness of the neural network.

**Note:** The optimal *num_hidden* : *num_visible* ratio will depend on the system. For the two-qubit wavefunction described above, good results are yielded when this ratio is 1.

On top of needing the number of visible and hidden units, a *ComplexWavefunction* object requires the user to input a dictionary containing the unitary operators (2x2) that will be used to rotate the qubits in and out of the computational basis, Z, during the training process. The *unitaries* utility will take care of creating this dictionary.

The *MetricEvaluator* class and *training_statistics* utility are built-in amenities that will allow the user to evaluate the training in real time. 


### Training
To evaluate the training in real time, the fidelity between the true wavefunction of the system and the wavefunction that qucumber reconstructs, $\vert\langle\psi\vert\psi_{RBM}\rangle\vert^2$, will be calculated along with the Kullback-Leibler (KL) divergence (the RBM's cost function). First, we need to load our training data and the true wavefunction of this system using the *data* utility.

In [2]:
train_path       = "qubits_train.txt"
train_bases_path = "qubits_train_bases.txt"
psi_path         = "qubits_psi.txt"
bases_path       = "qubits_bases.txt"

train_samples, true_psi, train_bases, bases = data.load_data(train_path, psi_path, train_bases_path, bases_path)

As previouosly mentioned, a *ComplexWavefunction* object requires a dictionary that contains the unitariy operators that will be used to rotate the qubits in and out of the computational basis, Z, during the training process. In the case of the provided dataset, the unitaries required are the well-known $H$, and $K$ gates. The dictionary needed can be created with the following command.

In [3]:
unitary_dict = unitaries.create_dict()
#unitary_dict = unitaries.create_dict(name='your_name', 
#                                     unitary=torch.tensor([[real part], 
#                                                           [imaginary part]], 
#                                                          dtype=torch.double)

If the user wishes to add their own unitary operators from their experiment to *unitary_dict*, uncomment the block above. When *unitaries.create_dict()* is called, it will contain the identity, the $H$ and $K$ gates by default with the keys "Z", "X" and "Y", respectively.

The number of visible units in the RBM is equal to the number of qubits. The number of hidden units will also be taken to be the number of visible units.

In [4]:
nv = train_samples.shape[-1]
nh = nv

nn_state = ComplexWavefunction(num_visible=nv, num_hidden=nh, unitary_dict=unitary_dict)
#nn_state = ComplexWavefunction(num_visible=nv, num_hidden=nh, unitary_dict=unitary_dict, gpu=False)

By default, qucumber will attempt to run on a GPU if one is available (if one is not available, qucumber will default to CPU). If one wishes to run qucumber on a CPU, add the flag "gpu = False" in the *ComplexWavefunction* object instantiation (i.e. uncomment the line above). 

Now we can specify the hyperparameters of the training process. 

1. **epochs**: the total number of training cycles that will be performed (default = 100)
2. **pos_batch_size**: the number of data points used in the positive phase of the gradient (default = 100)
3. **neg_batch_size**: the number of data points used in the negative phase of the gradient (default = *pos_batch_size*)
4. **k**: the number of contrastive divergence steps (default = 1)
5. **lr**: the learning rate (default = 0.001)

    **Note:** For more information on the hyperparameters above, we strongly encourage the user to read through our brief, but thorough theory document on RBMs. One does not have to specify these hyperparameters, as their default values will be used without the user overwriting them. It is recommended to keep with the default values until the user has a stronger grasp on what these hyperparameters mean. The quality and the computational efficiency of the training will highly depend on the choice of hyperparameters. As such, playing around with the hyperparameters is almost always necessary. 
    
The two-qubit example in this tutorial should be extremely easy to train, regardless of the choice of hyperparameters. However, the hyperparameters below will be used.

In [5]:
epochs = 300
pbs    = 50 # pos_batch_size
nbs    = 10 # neg_batch_size
lr     = 0.01
k      = 1

For evaluating the training in real time, we will call on the *MetricEvaluator* to calculate the training evaluators every 10 epochs. The *MetricEvaluator* requires the following arguments.

1. **log_every**: the frequency of the training evaluators being calculated is controlled by the *log_every* argument (e.g. *log_every* = 200 means that the *MetricEvaluator* will update the user every 200 epochs)
2. A dictionary of functions you would like to reference to evaluate the training (arguments required for these functions are keyword arguments placed after the dictionary)

The following additional arguments are needed to calculate the fidelity and KL divergence in the *training_statistics* utility.

- **target_psi** (the true wavefunction of the system)
- **verbose** (a boolean for telling the *MetricEvaluator* to print the training evaluators)
- **space** (the hilbert space of the system)

In [6]:
log_every = 50
nn_state.space = nn_state.generate_hilbert_space(nv)

callbacks = [
    MetricEvaluator(log_every, {"Fidelity": ts.fidelity, "KL": ts.KL}, target_psi=true_psi, bases=bases,
                    verbose=True, space=nn_state.space)
]

Now we can begin training. The *PositiveWavefunction* object has a property called *fit* which takes care of this for us.

In [7]:
nn_state.fit(train_samples, epochs=epochs, pos_batch_size=pbs, neg_batch_size=nbs, lr=lr, k=k, input_bases=train_bases,
             callbacks=callbacks)

Epoch: 50	Fidelity = 0.754661	KL = 0.134845


KeyboardInterrupt: 

It should be noted that all the code past the fourth code block of this notebook is not needed for the training process. One could have just ran *nn_state.fit(train_samples)* and just used the default hyperparameters and no training evaluators.

### Post-training
#### Saving and loading parameters

At the end of the training process, the network parameters (the weights, visible biases and hidden biases) are stored in the *ComplexWavefunction* object. One can save them to a pickle file, which will be called *saved_params*, with the following command.

In [ ]:
nn_state.save("saved_params.pt")

This saves the weights, visible biases and hidden biases as torch tensors with the following keys: "weights", "visible_bias", "hidden_bias". One can then load a new *ComplexWavefunction* object with saved parameters with the following command.

In [ ]:
nn_state2 = ComplexWavefunction.autoload("saved_params.pt")

#### Generate new data

At the end of the training process, the user can then generate new data. A *ComplexWavefunction* object has a property called *sample* that takes in the following arguments.

1. **k**: the number of Gibbs steps to perform to generate the new samples
2. **num_samples**: the number of new data points to be generated

In [ ]:
new_samples = nn_state.sample(k=10, num_samples=100)

This new data can also be saved (along with the RBM parameters) with the following command.

In [ ]:
nn_state.save("saved_params_and_new_data.pt", metadata={"samples":new_samples})

*metadata* is a disctionary that is an optional argument in the *PositiveWavefunction*'s *save* function.